# Text Analysis

 In this lecture, we look at more recent methods of feature extraction and topic modeling.

We will cover the following:

- word2vec
- latent semantic analysis
- non-negative matrix factorization
- latent Dirichlet allocation

A technical guide to topic modeling can be found in these [lecture notes](http://pages.cs.wisc.edu/~jerryzhu/cs769/latent.pdf) but is outside the scope of this class.

## Similarity

In order to find similar words or documents after they have been vectorized, we need definitions of similarity. Similarity measures often used in text analysis include

- edit 
- cosine 
- Hellinger 
- Kullback-Leibler 
- Jacard 

These may be given as the similarity or distance.

### Edit 

The edit distance between two strings is the minimum number of changes needed to covert from one string to another. These changes may be weighted, for example, by making a deletion changes have a different weight than an insertion operation. Also known as Levenshtein distance.

Such distance metrics are the basis for aligning DNA, RNA and protein sequences.

In [ ]:
! python3 -m pip install --quiet textdistance

In [ ]:
import textdistance as td

In [ ]:
td.levenshtein.distance('slaves', 'salve')

In [ ]:
td.levenshtein.similarity('slaves', 'salve')

### Jacard 

The Jacard distance is the intersection divided by union of two sets.

In [ ]:
td.jaccard.similarity('the quick brown fox'.split(), 'the quick brown dog'.split())

Note that the implementation is actually for multisets.

In [ ]:
td.jaccard.similarity('slaves', 'salve')

### Cosine 

For two real valued vectors.

In [ ]:
s1 = 'the quick brown fox'
s2 = 'the quick brown dog'

In [ ]:
td.cosine.similarity(s1.split(), s2.split())

Cosine distance works on vectors - the default is just to use the bag of words counts.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
t = cv.fit_transform([s1, s2]).toarray()
t

Cosine distance is equivalent to the inner product of the normalized vectors with length 1.

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
import numpy as np

In [ ]:
np.around(1- cosine(t[0], t[1]), 2)

In [ ]:
np.dot(t[0]/np.linalg.norm(t[0]), t[1]/np.linalg.norm(t[1]))

### Hellinger

For two probability distributions.

In [ ]:
from gensim.matutils import hellinger

In [ ]:
p = t[0]/t[0].sum()
q = t[1]/t[1].sum()
p, q

In [ ]:
hellinger(p, q)

In [ ]:
def discrete_hellinger(p, q):
    return 1/np.sqrt(2) * np.linalg.norm(np.sqrt(p) - np.sqrt(q))

In [ ]:
discrete_hellinger(p, q)

### Kullback-Leibler

In [ ]:
t = cv.fit_transform(['one two three', 'one one one two two three']).toarray()
t

In [ ]:
p = t[0]/t[0].sum()
q = t[1]/t[1].sum()
p, q

In [ ]:
from gensim.matutils import  kullback_leibler

In [ ]:
kullback_leibler(p, q)

Not symmetric.

In [ ]:
kullback_leibler(q, p)

In [ ]:
def discrete_dkl(p, q):
    return -np.sum(p * (np.log(q) - np.log(p)))

In [ ]:
discrete_dkl(p, q)

In [ ]:
discrete_dkl(q, p)

## Word2Vec

the `word2vec` family of algorithms is a powerful method for converting a word into a vector that takes into account its context. There are two main ideas - in continuous bag of words, we try to predict the current word from nearby words; in continuous skip-gram, the current word is used to predict nearby words. The phrase "nearby words" is intentionally vague - in the simplest case, it is a sliding window of words centered on the current word. 

Suppose we have the sentence

```
I do not like green eggs and ham
```

and suppose we use a centered window of length 3,

```
((I, not), do), ((do, like), not), ((not, green), like), ((like, eggs), green), ((green, and), eggs), ((eggs, ham) and)
```

In continuous bag of words, we make the (input, output) pairs to be
```
(I, do)
(not, do)
(do, not)
(like, not)
(not, like)
(green, like)
(like, green)
(eggs, green)
(green, eggs)
(and, eggs)
(eggs, and)
(ham, and)
```

That is, we try to predict `do` when we see `I`, `do` when we see `not` and so on.

In continuous skip-gram, we do the inverse for (input, output) pairs
```
(do, I)
(do, not)
(not, do)
(not, like)
(like, not)
(like, green)
(green, like)
(green, eggs)
(eggs, green)
(eggs, and)
(and, eggs)
(and, ham)
```

That is, we try to predict `I` when we see `do`, `not` when we see `do` and so on.

To do this prediction, we first assign each word to a vector of some fixed length $n$ - i.e. we embed each word as an $\mathbb{R}^n$ vector. To do a prediction for all words in the vocabulary using `softmax` would be prohibitively expensive, and is unnecessary if we are just trying to find a good embedding vector. Instead we select $k$ noise words, typically from the unigram distributions, and just train the classifier to distinguish the target word from the noise words using logistic regression (negative sampling). We use stochastic gradient descent to move the embedding word vectors (initialized randomly) until the model gives a high probability to the target words and low probability to the noise ones. If successful, words that are meaningful when substituted in the same context will be close together in $\mathbb{R}^n$. For instance, `dog` and `cat` are likely to be close together because they appear together in similar contexts like

- `My pet dog|cat`
- `Raining dogs|cats and cats|dogs`
- `The dog|cat chased the rat`
- `Common pets are dogs|cats`

while `dog` and `apple` are less likely to occur in the same context and hence will end up further apart in the embedding space. Interestingly, the vectors resulting from vector subtraction are also meaningful since they represent analogies - the vector between `man` and `woman` is likely to be similar to that between `king` and `queen`, or `boy` and `girl`.

Note: you will encounter `word2vec` again if you take a deep learning class - it is a very influential idea and has many applications beyond text processing since you can apply it to any discrete distribution where local context is meaningful (e.g. genomes). 

There is a very nice tutorial on Word2Vec that you should read if you want to learn more about the algorithm - [Part 1](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) and [Part 2](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)

Word2Vec learns about the feature representations of *words* - there is a an extension `doc2vec` that generates a feature vector for paragraphs or documents in the same way; we may cover this in the next lecture along with other document retrieval algorithms.

There are several other word to vector algorithms inspired by `word2vec` - for example, [`fasttext`](https://fasttext.cc), [approximate nearest neighbors](https://github.com/spotify/annoy) and `wordrank`. Conveniently, many of these are available in the `gensim.models` package.

We illustrate the mechanics of `word2vec` using `gensim` on the tiny newsgroup corpora; however, you really need much large corpora for `word2vec` to learn effectively.

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.collocations import QuadgramCollocationFinder, TrigramCollocationFinder
from nltk.metrics.association import QuadgramAssocMeasures, TrigramAssocMeasures
import string

In [ ]:
import gensim
from gensim.models.word2vec import Word2Vec

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
import warnings

warnings.simplefilter('ignore', FutureWarning)

In [ ]:
newsgroups_train = fetch_20newsgroups(
        subset='train',
        remove=('headers', 'footers', 'quotes')
)

In [ ]:
newsgroups_test = fetch_20newsgroups(
        subset='test',
        remove=('headers', 'footers', 'quotes')
)

In [ ]:
def gen_sentences(corpus):
    for item in corpus:
        yield from nltk.tokenize.sent_tokenize(item)

In [ ]:
for i, t in enumerate(newsgroups_train.target[:20]):
    print('%-24s:%s' % (newsgroups_train.target_names[t], 
                        newsgroups_train.data[i].strip().replace('\n', ' ')[:50]))

In [ ]:
list(newsgroups_train.data[:3])

In [ ]:
list(gen_sentences(newsgroups_train.data[:2]))[:3]

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
docs = [gensim.utils.simple_preprocess(s) 
        for s in newsgroups_train.data]
docs = [[s for s in doc if not s in STOPWORDS] for doc in docs]

In [ ]:
try:
    model = Word2Vec.load('newsgroup_w2v.model')
except:
    model = Word2Vec(docs,
                     vector_size=64, # we use 64 dimensions to represent each word
                     window=5, # size of each context window
                     min_count=3, # ignore words with frequency less than this
                     workers=4)
    model.train(docs, total_examples=len(docs), epochs=10)
    model.save('newsgroup_w2v.model')

In [ ]:
len(model.wv.key_to_index)

The embedding vector for the word `player`

In [ ]:
model.wv.get_vector('england')

In [ ]:
model.wv.most_similar('england', topn=5)

In [ ]:
model.wv.similarity('england', 'france')

In [ ]:
model.wv.similarity('england', 'rabbit')

Apparently, man is to baseball as woman is to stats. Who knew?

In [ ]:
model.wv.most_similar(positive=['baseball', 'man'], negative=['woman'], topn=3)

Because of the small and very biased data sets (including `soc.religion.christian` and `alt.atheism`), some of the analogies found are pretty weird.

In [ ]:
model.wv.most_similar(positive=['father', 'son'],
                   negative=['mother'])

## Doc2Vec

The `doc2vec` algorithm is basically the same as `word2vec` with the addition of a paragraph or document context vector. That is, certain words may be used differently in different types of documents, and this is captured in the  vector representing the paragraph or document.

![img](https://cdn-images-1.medium.com/max/1600/0*x-gtU4UlO8FAsRvL.)

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
tagged_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]

In [ ]:
try:
    model = Doc2Vec.load('newsgroup_d2v.model')
except:
    model = Doc2Vec(tagged_docs,
                    vector_size=10, # we use 10 dimensions to represent each doc
                    window=5, # size of each context window
                    min_count=3, # ignore words with frequency less than this
                    workers=4)
    model.train(tagged_docs, total_examples=len(tagged_docs), epochs=10)
    model.save('newsgroup_d2v.model')

In [ ]:
query = newsgroups_test.data[9]

In [ ]:
query = [token for token in gensim.utils.simple_preprocess(query) 
         if not token in STOPWORDS]

In [ ]:
vector = model.infer_vector(query)
vector

In [ ]:
model.docvecs.most_similar([vector])

In [ ]:
print(newsgroups_test.data[9])
for i, score in model.docvecs.most_similar([vector], topn=5):
    print('-'*80)
    print(newsgroups_train.data[i])

## Latent Semantic Indexing (LSI)

### Concept

Latent semantic indexing is basically using SVD to find a low rank approximation to the document/word feature matrix.

Recall that with SVD, $A = U \Sigma V^T$. With LSI, we interpret the matrices as

\begin{array}
& A &= & T & \Sigma & D^T  \\
(t \times d) &= & (t \times n) & (n \times n) & (n \times d)
\end{array}

where $T$ is a mnemonic for Term and $D$ is a mnemonic for Document.

If we use $r$ singular values, we reconstruct the rank-$r$ matrix $A_r$ as 

\begin{array}
& A_r &= & \hat{T} & \hat{\Sigma} & \hat{D}^T  \\
(t \times d) &= & (t \times r) & (r \times r) & (r \times d)
\end{array}

or as the sum of outer products

$$
A_r = \sum_{k=1}^{r} \sigma_r t_r d_r^T
$$

The $r$ columns $\hat{T}$ are the basis vectors for the rotated lower-dimensional coordinate system, and we can consider each of the $r$ columns or $\hat{T}$ as representing a topic. The value of $\hat{T}_{ij}$ is the weight of the $i^\text{th}$ term for topic $j$.

### Queries

Suppose we have a new document $x$ with dimensions $t \times 1$. We convert it to the $\hat{T}$ space by a change-of basis transformation

$$
x^* = \hat{T}^T x
$$

which you can check will have dimensions $r \times 1$.

To find what documents are similar to $x$, we look for what original documents are close to $x^*$ in the $\hat{T}$ space by looking for the columns of $\hat{\Sigma} D^T$ (with dimensions $r \times d$) that are closest to $x*$.

### Example of LSI

In [ ]:
len(docs)

In [ ]:
dictionary = gensim.corpora.Dictionary(docs)

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
lsi = gensim.models.LsiModel(corpus, num_topics=10, id2word = dictionary)

In [ ]:
for i, topic in  lsi.print_topics(num_words=5):
    print(topic)

In [ ]:
for i, t in enumerate(newsgroups_test.target[:20]):
    print('%02d %-24s:%s' % (i, newsgroups_test.target_names[t], 
                        newsgroups_test.data[i].strip().replace('\n', ' ')[:50]))

#### Find topics in document

Note that topics are rather hard to interpret. After all they are just words with the largest weights in the low rank approximation.

In [ ]:
query = newsgroups_test.data[9]
query

In [ ]:
query = gensim.utils.simple_preprocess(query)
query[:3]

In [ ]:
query = dictionary.doc2bow(query)

In [ ]:
query = lsi[query]

In [ ]:
sorted(query, key=lambda x: -x[1])[:5]

In [ ]:
topics = [i for i, score in sorted(query, key=lambda x: -x[1])[:5]]

In [ ]:
lsi.print_topic(topics[0])

In [ ]:
pat = re.compile(r'.*?(-)?\d+.*?\"(\w+)\"')

In [ ]:
for topic in topics:
    words = [''.join(pair) for pair in pat.findall(lsi.print_topic(topic))]
    print(','.join(words))

#### Find similar documents

In [ ]:
index = gensim.similarities.MatrixSimilarity(lsi[corpus])

In [ ]:
sims = index[query]

In [ ]:
hits = sorted(enumerate(sims), key=lambda x: -x[1])[:5]
hits

In [ ]:
print(newsgroups_test.data[9])
for match in [newsgroups_train.data[k] for k, score in hits]:
    print('-'*80)
    print(match)

$$
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

## Non-negative Matrix Factorization

The topics generated by LSI can be hard to understand because they include negative weights for words. They may also be hard to understand since they may not map to topics in the way that we would. Remember, a topic is just a low rank approximation that minimizes the Frobenius norm. An alternative factorization is non-negative matrix (NMF) factorization, which does not use negatively valued words in the topic.

NMF performs the following decomposition

\begin{array}
& A &= & W & H  \\
(t \times d) &= & (t \times n) & (n \times d)
\end{array}

using an iterative procedure to minimize the Frobenius norm $\norm{A - WH}_F^2$ subject to the constraint that $W, H > 0$. There are several different methods to perform this iterative minimization that do not concern us here.

NMF basically finds a different set of basis vectors (not the eigenvectors of the covariance matrix) to project onto. The vectors point in the direction of clusters of word features that appear in common across multiple documents.

![nmf_svd](https://qph.fs.quoracdn.net/main-qimg-9b4e31ec4b57f4baf7d08d5df17c6bc0)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [ ]:
import warnings

In [ ]:
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups_train.data)
X = normalize(X, norm='l1', axis=1)
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(X)

In [ ]:
W.shape

In [ ]:
vocab = vectorizer.get_feature_names()
for i, topic in enumerate(model.components_):
    print("Topic %d:" % i, end=' ')
    print(" ".join([vocab[i] for i in topic.argsort()[:-10 - 1:-1]]))

How would you find documents similar to the query document? 

## Latent Dirichlet Allocation

- [Original paper](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)

In [ ]:
import numpy as np

### The Dirichlet distribution

A random sample from a Dirichlet distribution is a multinomial distribution, of the same length as the Dirichlet concentration parameter $\alpha$

In [ ]:
α = np.array([1,2,3])
for i in range(5):
    print(np.random.dirichlet(α))

#### Relationship between $\alpha$ and samples

In [ ]:
α/α.sum()

In [ ]:
n = int(1e6)
np.random.dirichlet(α, n).mean(axis=0)

### Concept of LDA

LDA is a generative model - that is, it provides a probability distribution from which we can generate documents, each of which is composed of generated words. We sketch the generative process here; the MCMC machinery that is used for implementation is not covered in this course (but will be in STA 663).

- There are $M$ documents
  - A document consists of the words $w_{1:N}$
- There are $K$ topics $\varphi_{1:K}$ from which we can choose words from a vocabulary of length $V$
  - For each topic
    - Sample a topic $\varphi$ from a Dirichlet distribution with parameter $\beta$
    - Each topic $\varphi$ is a multinomial distribution of size $V$
- There are $N$ words in a document
  - For each document
    - Sample a topic multinomial $\theta$ of size $K$ from a different Dirichlet distribution with parameter $\alpha$
    - Repeat for each word position in the document
      - Sample the integer index $z$ from $\theta$
      - Sample a word $w$ from the topic $\varphi_z$ 
      
![lda](https://upload.wikimedia.org/wikipedia/commons/4/4d/Smoothed_LDA.png)

### Example of LDA

In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
lda = LdaModel(corpus, num_topics=10, id2word = dictionary)

In [ ]:
for i, topic in  lda.print_topics(num_words=5):
    print(topic)

In [ ]:
query = newsgroups_test.data[9]
query

In [ ]:
query = gensim.utils.simple_preprocess(query)
query[:3]

In [ ]:
query = dictionary.doc2bow(query)

In [ ]:
query = lda[query]

#### Topics in query document

In [ ]:
sorted(query, key=lambda x: -x[1])[:5]

In [ ]:
index = gensim.similarities.MatrixSimilarity(lda[corpus])

In [ ]:
sims = index[query]

In [ ]:
topics = [i for i, score in sorted(query, key=lambda x: -x[1])[:5]]

In [ ]:
lda.print_topic(topics[0])

#### Find similar documents

In [ ]:
hits = sorted(enumerate(sims), key=lambda x: -x[1])[:5]
hits

In [ ]:
print(newsgroups_test.data[9])
for match in [newsgroups_train.data[k] for k, score in hits]:
    print('-'*80)
    print(match)